# Quality Assessment
For this part, we use *recording channel amplitude vector* which is a n_channnels×1  positive  vector  of  the  robust  standard  deviations (1.4826 times of the  median  absolute  deviation  from  the  sample median) of the filtered channel signals from channels (Shamlo et al., 2020).

Due to the robust estimators being used, these measures are partially biased towards brain signals rather than  artifacts, and can thus be used to track impacts on those brain signals before and after a given pre-processing method is applied (Robbins et al., 2020).

In particular, we aim at assessing the quality of data before and after Autoreject and ICA in order to make an informed decision about necessity of running ICA in the pipeline for eliminating ocular artefacts in our data.

## Setup

In [2]:
# Import necessary modules
import mne
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import mne_bids
from pathlib import Path
from autoreject import AutoReject
from autoreject import get_rejection_threshold
from pyprep.prep_pipeline import PrepPipeline
from scipy.stats import median_abs_deviation



# utility function transalted from Vislab's MATLAB code to python
def dispersion_with_mean(matrix):
    """
    calculate dispersion vector [1]
    this function use mean for normalization

    Parameters
    ----------
    matrix : array-like
        amplitude matrix. rows are channels and columns are recordings

    Reference
    ---------
    [1] Bigdely-Shamlo, N., Touryan, J., Ojeda, A., Kothe, C., Mullen, T., & Robbins, K. (2020).
    Automated EEG mega-analysis I: Spectral and amplitude characteristics across studies. NeuroImage, 207, 116361.

    """
    mean_amp = matrix.mean(1)
    matrix_normalized = matrix / mean_amp
    stdFromMad = 1.4826 * median_abs_deviation(matrix_normalized,1)
    dispersion = stdFromMad / np.median(matrix_normalized); 

    return dispersion

def make_continious_data(subject, bids_root=Path('data/Main-study'), tasks='all'):
    """
    This function concatenate recordings of one subject in bids format

    """
    session = '01'
    if tasks == 'all':
        tasks = [ 'baseline1',
            'induction1', 'experience1', 
            'induction2', 'experience2',
            'induction3', 'experience3',
            'induction4', 'experience4',
            'baseline2'
        ]

    # open baseline1
    bids_path = mne_bids.BIDSPath(subject=subject, session=session, task='baseline1', root=bids_root)
    raw = mne_bids.read_raw_bids(bids_path, verbose=False)

    for task in tasks:
        bids_path = mne_bids.BIDSPath(subject=subject, session=session, task=task, root=bids_root)
        raw_temp = mne_bids.read_raw_bids(bids_path, verbose=False)
        raw.append(raw_temp)
    
    return raw

def make_amplitude_matrix(ampVectors):
    """
    ampVectors : list of np.array
    
    """
    amplitude_matrix = [
        np.append(np.zeros(ampVectors[0].shape), ampVectors[i])
        for i in range(len(ampVectors))
    ]
    amplitude_matrix = np.delete(amplitude_matrix, 0, axis=0)
    return amplitude_matrix
    

In [ ]:
# raw = mne.io.read_raw_brainvision('data/raw/plb-hyp-live2131111.vhdr', misc=['ECG'], preload=True, verbose=False)
# raw.set_channel_types({'ECG':'ecg'})
subject='01'
session = '01'
task = 'experience1'
bids_root = '/Users/yeganeh/Codes/otka-preprocessing/data/Main-study'
bids_path = mne_bids.BIDSPath(subject=subject, session=session, task=task, root=bids_root)
sampling_freq = 1000

raw = mne_bids.read_raw_bids(bids_path, extra_params={'preload':True}, verbose=False) 
raw.crop(100,300)
raw.set_eeg_reference()
raw.filter(1,20)
raw.drop_channels(['ECG', 'EOG1', 'EOG2'])
# Calculate robust std from MAD
ampvector = 1.4826 * median_abs_deviation(raw.get_data(),1)

pos = make_montage()
raw.set_montage(pos)
pos = raw.get_montage()

ch_pos = np.array(list(pos.get_positions()['ch_pos'].values())) 
ch_pos = ch_pos[:,0:2]

mne.viz.plot_topomap(ampvector, ch_pos)